In [2]:
import pandas as pd
import numpy as np
import sys

In [3]:
BASE_MAXQUANT_TXT_DIR = '/Users/darylwilding-mcbride/Downloads/maxquant_results/txt'
ALLPEPTIDES_FILENAME = '{}/allPeptides.txt'.format(BASE_MAXQUANT_TXT_DIR)
PASEF_MSMS_SCANS_FILENAME = '{}/pasefMsmsScans.txt'.format(BASE_MAXQUANT_TXT_DIR)

In [4]:
RT_LIMIT_LOWER = 3000  # RT range in the database
RT_LIMIT_UPPER = 3060

In [5]:
pasef_msms_scans_df = pd.read_csv(PASEF_MSMS_SCANS_FILENAME, sep='\t')

In [6]:
pasef_msms_scans_df.head()

,Raw file,Frame,Precursor,IsolationMz,IsolationWidth,ScanNumBegin,ScanNumEnd,CollisionEnergy,Index
0,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,1,742.744835,2.427448,507,532,36.96,0
1,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,2,709.235759,2.092358,549,574,36.96,1
2,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,3,594.235088,2.000000,602,627,36.96,2
3,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,4,503.527768,2.000000,643,668,36.96,3
4,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,5,552.063870,2.000000,684,709,31.92,4


In [7]:
allpeptides_df = pd.read_csv(ALLPEPTIDES_FILENAME, sep='\t')
allpeptides_df.rename(columns={'Number of isotopic peaks':'isotope_count', 'm/z':'mz', 'Number of data points':'number_data_points', 'Intensity':'intensity', 'Ion mobility index':'scan', 'Ion mobility index length':'scan_length', 'Ion mobility index length (FWHM)':'scan_length_fwhm', 'Retention time':'rt', 'Retention length':'rt_length', 'Retention length (FWHM)':'rt_length_fwhm', 'Charge':'charge_state', 'Number of pasef MS/MS':'number_pasef_ms2_ids', 'Pasef MS/MS IDs':'pasef_msms_ids', 'MS/MS scan number':'msms_scan_number', 'Isotope correlation':'isotope_correlation'}, inplace=True)
allpeptides_df = allpeptides_df[allpeptides_df.intensity.notnull() & allpeptides_df.pasef_msms_ids.notnull()].copy()
allpeptides_df.msms_scan_number = allpeptides_df.msms_scan_number.apply(lambda x: int(x))

In [8]:
allpeptides_df['pasef_msms_ids_list'] = allpeptides_df.pasef_msms_ids.str.split(";").apply(lambda x: [int(i) for i in x])

In [9]:
allpeptides_df.head()

,Raw file,charge_state,mz,Mass,Resolution,number_data_points,Number of frames,isotope_count,isotope_correlation,Mass fractional part,...,Max frame index,scan,scan_length,scan_length_fwhm,intensity,Intensities,number_pasef_ms2_ids,pasef_msms_ids,msms_scan_number,pasef_msms_ids_list
71,HeLa_20KInt_2KIT_Slot1-46_01_1179,1,336.92138,335.91410,24300.250397,1532,22,2,0.991542,0.914104,...,5556,726,24,12,12269.0,NaN,3,483530;483531;483532,-1,"[483530, 483531, 483532]"
73,HeLa_20KInt_2KIT_Slot1-46_01_1179,1,336.92118,335.91390,23643.933610,1035,16,2,0.997353,0.913903,...,7537,729,18,15,7415.5,NaN,7,486675;486676;486677;486678;486679;486680;486681,22,"[486675, 486676, 486677, 486678, 486679, 48668..."
74,HeLa_20KInt_2KIT_Slot1-46_01_1179,1,336.92070,335.91342,24001.610890,1067,21,2,0.995011,0.913421,...,6180,726,18,9,8451.1,NaN,8,485479;485480;485481;485482;485483;485484;4854...,17,"[485479, 485480, 485481, 485482, 485483, 48548..."
75,HeLa_20KInt_2KIT_Slot1-46_01_1179,1,331.88454,330.87726,24257.803013,2815,41,2,0.998650,0.877259,...,5618,732,15,9,19442.0,NaN,12,483717;483719;483720;483721;483722;483855;4838...,7,"[483717, 483719, 483720, 483721, 483722, 48385..."
76,HeLa_20KInt_2KIT_Slot1-46_01_1179,1,331.88465,330.87738,24175.211648,1255,24,2,0.990039,0.877375,...,7041,729,21,15,13239.0,NaN,13,486231;486232;486233;486234;486235;486299;4863...,9,"[486231, 486232, 486233, 486234, 486235, 48629..."


In [10]:
precursor_id = 93486

In [11]:
df = pasef_msms_scans_df[pasef_msms_scans_df.Precursor == precursor_id]
df

,Raw file,Frame,Precursor,IsolationMz,IsolationWidth,ScanNumBegin,ScanNumEnd,CollisionEnergy,Index
201178,HeLa_20KInt_2KIT_Slot1-46_01_1179,28320,93486,670.269329,2.0,530,555,36.96,201178
201187,HeLa_20KInt_2KIT_Slot1-46_01_1179,28321,93486,670.269329,2.0,530,555,36.96,201187
201196,HeLa_20KInt_2KIT_Slot1-46_01_1179,28323,93486,670.269329,2.0,530,555,36.96,201196


In [12]:
for i in range(len(df)):
    pasef_msms_id = pasef_msms_scans_df[pasef_msms_scans_df.Precursor == precursor_id].Index.iloc[i]
    print(allpeptides_df[allpeptides_df.pasef_msms_ids_list.apply(lambda x: pasef_msms_id in x)].mz)

172338    669.35749
Name: mz, dtype: float64
172338    669.35749
Name: mz, dtype: float64
172338    669.35749
Name: mz, dtype: float64


In [13]:
msms_scan_number = allpeptides_df[allpeptides_df.pasef_msms_ids_list.apply(lambda x: pasef_msms_id in x)].msms_scan_number.iloc[0]

In [14]:
msms_scan_number  # this is the 'index' that appears in the APL

39343

In [16]:
pasef_msms_scans_df[pasef_msms_scans_df.Index == 61385]

,Raw file,Frame,Precursor,IsolationMz,IsolationWidth,ScanNumBegin,ScanNumEnd,CollisionEnergy,Index
61385,HeLa_20KInt_2KIT_Slot1-46_01_1179,10253,27349,386.37078,2.0,754,779,31.92,61385


In [17]:
allpeptides_df.columns

Index(['Raw file', 'charge_state', 'mz', 'Mass', 'Resolution',
       'number_data_points', 'Number of frames', 'isotope_count',
       'isotope_correlation', 'Mass fractional part', 'Mass deficit', 'rt',
       'rt_length', 'rt_length_fwhm', 'Min frame index', 'Max frame index',
       'scan', 'scan_length', 'scan_length_fwhm', 'intensity', 'Intensities',
       'number_pasef_ms2_ids', 'pasef_msms_ids', 'msms_scan_number',
       'pasef_msms_ids_list'],
      dtype='object')

In [18]:
allpeptides_df[allpeptides_df.pasef_msms_ids_list.apply(lambda x: 61385 in x)]

,Raw file,charge_state,mz,Mass,Resolution,number_data_points,Number of frames,isotope_count,isotope_correlation,Mass fractional part,...,Max frame index,scan,scan_length,scan_length_fwhm,intensity,Intensities,number_pasef_ms2_ids,pasef_msms_ids,msms_scan_number,pasef_msms_ids_list
6646,HeLa_20KInt_2KIT_Slot1-46_01_1179,2,386.22759,770.44063,24169.279354,2949,21,3,0.948521,0.44063,...,943,762,45,21,31537.0,NaN,3,61378;61385;61393,4201,"[61378, 61385, 61393]"


In [19]:
allpeptides_df[allpeptides_df.msms_scan_number == 61385]

,Raw file,charge_state,mz,Mass,Resolution,number_data_points,Number of frames,isotope_count,isotope_correlation,Mass fractional part,...,Max frame index,scan,scan_length,scan_length_fwhm,intensity,Intensities,number_pasef_ms2_ids,pasef_msms_ids,msms_scan_number,pasef_msms_ids_list
255189,HeLa_20KInt_2KIT_Slot1-46_01_1179,2,815.44389,1628.8732,24044.569928,4743,24,3,0.986411,0.873225,...,2566,468,42,24,48732.0,NaN,3,198616;198653;198661,61385,"[198616, 198653, 198661]"


In [23]:
l = allpeptides_df[allpeptides_df.msms_scan_number == 61385].iloc[0].pasef_msms_ids_list

In [24]:
l

[198616, 198653, 198661]

In [28]:
pasef_msms_scans_df[pasef_msms_scans_df.Index.isin(l)]

,Raw file,Frame,Precursor,IsolationMz,IsolationWidth,ScanNumBegin,ScanNumEnd,CollisionEnergy,Index
198616,HeLa_20KInt_2KIT_Slot1-46_01_1179,28011,92316,815.920649,3.0,457,482,42.0,198616
198653,HeLa_20KInt_2KIT_Slot1-46_01_1179,28016,92316,815.920649,3.0,457,482,42.0,198653
198661,HeLa_20KInt_2KIT_Slot1-46_01_1179,28017,92316,815.920649,3.0,457,482,42.0,198661
